In [1]:
from google.colab import files
uploaded = files.upload()

Saving hmeq.csv to hmeq.csv


In [28]:
import pandas as pd
import shap
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier as xgb
from sklearn.ensemble import RandomForestClassifier as rfcs
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

In [3]:
data = pd.read_csv('hmeq.csv')
data

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,0,88900,57264.0,90185.0,DebtCon,Other,16.0,0.0,0.0,221.808718,0.0,16.0,36.112347
5956,0,89000,54576.0,92937.0,DebtCon,Other,16.0,0.0,0.0,208.692070,0.0,15.0,35.859971
5957,0,89200,54045.0,92924.0,DebtCon,Other,15.0,0.0,0.0,212.279697,0.0,15.0,35.556590
5958,0,89800,50370.0,91861.0,DebtCon,Other,14.0,0.0,0.0,213.892709,0.0,16.0,34.340882


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5960 entries, 0 to 5959
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAD      5960 non-null   int64  
 1   LOAN     5960 non-null   int64  
 2   MORTDUE  5442 non-null   float64
 3   VALUE    5848 non-null   float64
 4   REASON   5708 non-null   object 
 5   JOB      5681 non-null   object 
 6   YOJ      5445 non-null   float64
 7   DEROG    5252 non-null   float64
 8   DELINQ   5380 non-null   float64
 9   CLAGE    5652 non-null   float64
 10  NINQ     5450 non-null   float64
 11  CLNO     5738 non-null   float64
 12  DEBTINC  4693 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 605.4+ KB


In [5]:
data.describe()

,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
count,5960.000000,5960.000000,5442.000000,5848.000000,5445.000000,5252.000000,5380.000000,5652.000000,5450.000000,5738.000000,4693.000000
mean,0.199497,18607.969799,73760.817200,101776.048741,8.922268,0.254570,0.449442,179.766275,1.186055,21.296096,33.779915
std,0.399656,11207.480417,44457.609458,57385.775334,7.573982,0.846047,1.127266,85.810092,1.728675,10.138933,8.601746
min,0.000000,1100.000000,2063.000000,8000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.524499
25%,0.000000,11100.000000,46276.000000,66075.500000,3.000000,0.000000,0.000000,115.116702,0.000000,15.000000,29.140031
50%,0.000000,16300.000000,65019.000000,89235.500000,7.000000,0.000000,0.000000,173.466667,1.000000,20.000000,34.818262
75%,0.000000,23300.000000,91488.000000,119824.250000,13.000000,0.000000,0.000000,231.562278,2.000000,26.000000,39.003141
max,1.000000,89900.000000,399550.000000,855909.000000,41.000000,10.000000,15.000000,1168.233561,17.000000,71.000000,203.312149


In [6]:
missing = data.isnull().sum()
missing = missing[missing> 0].sort_values(ascending = False)
missing

,0
DEBTINC,1267
DEROG,708
DELINQ,580
MORTDUE,518
YOJ,515
NINQ,510
CLAGE,308
JOB,279
REASON,252
CLNO,222


In [7]:
x = data.drop('BAD', axis = 1)
y = data['BAD']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [12]:
print(data.select_dtypes(include=['int64', 'float64']).columns)
print(data.select_dtypes(include = 'object').columns)


Index(['BAD', 'LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE',
       'NINQ', 'CLNO', 'DEBTINC'],
      dtype='object')
Index(['REASON', 'JOB'], dtype='object')


In [19]:
num_cols = ['LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE',
       'NINQ', 'CLNO', 'DEBTINC']
cat_cols = ['REASON', 'JOB']

In [20]:
num_process = Pipeline([
    ('impute', SimpleImputer(strategy = 'mean')),
    ('scaler', StandardScaler())
])
cat_process = Pipeline([
    ('imputer', SimpleImputer(fill_value= 'missing', strategy = 'constant')),
    ('encoder', OneHotEncoder())
])
clm = ColumnTransformer([
    ('num', num_process, num_cols),
    ('cat', cat_process, cat_cols)
])

In [35]:
pipe = Pipeline([
    ('processor', clm),
    ('rfc', rfcs())
])
params = {
    'rfc__n_estimators': [100],
    'rfc__max_depth': [None, 10],
    'rfc__min_samples_split': [2],
    'rfc__min_samples_leaf': [1],
    'rfc__max_features': ['sqrt'],
    'rfc__bootstrap': [True],
    'rfc__criterion': ['gini']
}


In [36]:
grid = GridSearchCV(pipe, params, cv= 5, scoring = 'accuracy')

In [37]:
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('processor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['LOAN',
                                                                          'MORTDUE',
                                                                          'VALUE',
                                                                          'YOJ',
                                                                          'DEROG',
                                                                          'DELINQ',
                                                                          'CLAGE',
                                                                          'NINQ',
                                                                          'CLNO',
                                                                          'DEBTINC']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder())]),
                                                                         ['REASON',
                                                                          'JOB'])])),
                                       ('rfc', RandomForestClassifier())]),
             param_grid={'rfc__bootstrap': [True], 'rfc__criterion': ['gini'],
                         'rfc__max_depth': [None, 10],
                         'rfc__max_features': ['sqrt'],
                         'rfc__min_samples_leaf': [1],
                         'rfc__min_samples_split': [2],
                         'rfc__n_estimators': [100]},
             scoring='accuracy')

In [38]:
y_pred = grid.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       927
           1       0.90      0.71      0.79       265

    accuracy                           0.92      1192
   macro avg       0.91      0.84      0.87      1192
weighted avg       0.92      0.92      0.91      1192



In [40]:
from sklearn.linear_model import LogisticRegression
lr = Pipeline([
    ('processor', clm),
    ('logisticregression', LogisticRegression())
])

lr_params = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__solver': ['liblinear'] # 'liblinear' supports both l1 and l2 penalties
}
lrgrid = GridSearchCV(lr, lr_params, cv = 3, scoring = 'accuracy')
lrgrid.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('processor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['LOAN',
                                                                          'MORTDUE',
                                                                          'VALUE',
                                                                          'YOJ',
                                                                          'DEROG',
                                                                          'DELINQ',
                                                                          'CLAGE',
                                                                          'NINQ',
                                                                          'CLNO',
                                                                          'DEBTINC']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder())]),
                                                                         ['REASON',
                                                                          'JOB'])])),
                                       ('logisticregression',
                                        LogisticRegression())]),
             param_grid={'logisticregression__C': [0.001, 0.01, 0.1, 1, 10,
                                                   100],
                         'logisticregression__penalty': ['l1', 'l2'],
                         'logisticregression__solver': ['liblinear']},
             scoring='accuracy')

In [41]:
y_lr = lrgrid.predict(x_test)
print(classification_report(y_test, y_lr))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89       927
           1       0.68      0.31      0.42       265

    accuracy                           0.81      1192
   macro avg       0.75      0.63      0.66      1192
weighted avg       0.80      0.81      0.79      1192



In [54]:
from xgboost import XGBClassifier as xgb
pipe_xgb = Pipeline([
    ('processor', clm),
    ('xgb', xgb())
])

xgb_params = {
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [3, 5],
    'xgb__learning_rate': [0.1],
    'xgb__subsample': [1.0],
    'xgb__colsample_bytree': [1.0]
}

xgbgrid = GridSearchCV(pipe_xgb, xgb_params, cv = 3, scoring = 'accuracy')
xgbgrid.fit(x_train, y_train)

ValueError: 
All the 12 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 588, in _fit

In [46]:
y_xgb = xgbgrid.predict(x_test)
print(classification_report(y_test, y_xgb))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95       927
           1       0.87      0.77      0.82       265

    accuracy                           0.92      1192
   macro avg       0.90      0.87      0.89      1192
weighted avg       0.92      0.92      0.92      1192



In [53]:
import sklearn.compose._column_transformer as ct
import joblib
import pickle

# Patch for _RemainderColsList
if not hasattr(ct, "_RemainderColsList"):
    class _RemainderColsList(list):
        pass
    ct._RemainderColsList = _RemainderColsList

# Now safely load
model = joblib.load("xgb_credit_model.pkl")


In [52]:
from google.colab import files
files.download('xgb_credit_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>